<a href="https://colab.research.google.com/github/kridtapon/SlopePPO-Momentum/blob/main/SlopePPO_Momentum.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install --upgrade yfinance

In [2]:
pip install vectorbt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.6/527.6 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.7/295.7 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 18.5 MB/s eta 0:00:00


In [5]:
import numpy as np
import pandas as pd
import yfinance as yf
import vectorbt as vbt
from sklearn.linear_model import LinearRegression

# Function to calculate Exponential Moving Average (EMA)
def calculate_ema(series, span=200):
    return series.ewm(span=span, adjust=False).mean()

# Function to calculate Hull Moving Average (HMA)
def calculate_hma(series, window=200):
    wma1 = series.rolling(window=window//2).mean()
    wma2 = series.rolling(window=window).mean()
    hma = (2 * wma1 - wma2).rolling(window=int(np.sqrt(window))).mean()
    return hma

# Function to calculate Percentage Price Oscillator (PPO)
def calculate_ppo(close, fast_span=12, slow_span=26):
    fast_ema = calculate_ema(close, span=fast_span)
    slow_ema = calculate_ema(close, span=slow_span)
    ppo = (fast_ema - slow_ema) / slow_ema * 100  # PPO as a percentage
    return ppo

# Function to calculate the Signal Line for PPO
def calculate_ppo_signal(ppo, signal_span=9):
    return ppo.ewm(span=signal_span, adjust=False).mean()

# Function to calculate linear regression trendline projection
def calculate_trendline_projection(df, window=200):
    slope = np.full(len(df), np.nan)

    for i in range(window, len(df)):
        y = df['Close'][i-window:i].values.reshape(-1, 1)
        x = np.arange(0, window).reshape(-1, 1)

        model = LinearRegression().fit(x, y)
        slope[i] = model.coef_[0][0]  # Get the slope of the linear regression

    return pd.Series(slope, index=df.index)

# Define the stock symbol and time period
symbol = 'ANET'
start_date = '2019-01-01'
end_date = '2025-01-01'

# Download the data
df = yf.download(symbol, start=start_date, end=end_date)
df.columns = ['Close', 'High', 'Low', 'Open', 'Volume']

# Calculate 200 HMA for trend direction
df['HMA'] = calculate_hma(df['Close'], window=200)

# Calculate the Linear regression slope
df['Slope'] = calculate_trendline_projection(df, window=200)

# Calculate PPO and PPO Signal line
df['PPO'] = calculate_ppo(df['Close'])
df['PPO_Signal'] = calculate_ppo_signal(df['PPO'])

# Define Entry and Exit signals based on PPO and Moving Average Slope
df['Entry'] = (df['Slope'] > 0) & (df['PPO'] > df['PPO_Signal']) & (df['Close'] > df['HMA'])
df['Exit'] = (df['Slope'] < 0) & (df['PPO'] < df['PPO_Signal']) & (df['Close'] < df['HMA'])

# Filter data for the test period (2020-2025)
df = df[(df.index.year >= 2020) & (df.index.year <= 2025)]

# Backtest using vectorbt
portfolio = vbt.Portfolio.from_signals(
    close=df['Close'],
    entries=df['Entry'],
    exits=df['Exit'],
    init_cash=100_000,
    fees=0.001
)

# Display performance metrics
print(portfolio.stats())

# Plot equity curve
portfolio.plot().show()

[*********************100%***********************]  1 of 1 completed
/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sharpe_ratio' requires frequency to be set

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'calmar_ratio' requires frequency to be set

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'omega_ratio' requires frequency to be set

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sortino_ratio' requires frequency to be set



Start                         2020-01-02 00:00:00
End                           2024-12-31 00:00:00
Period                                       1258
Start Value                              100000.0
End Value                           173138.386989
Total Return [%]                        73.138387
Benchmark Return [%]                   763.853053
Max Gross Exposure [%]                      100.0
Total Fees Paid                         5818.9415
Max Drawdown [%]                        33.324581
Max Drawdown Duration                       314.0
Total Trades                                   24
Total Closed Trades                            23
Total Open Trades                               1
Open Trade PnL                       13072.582205
Win Rate [%]                            47.826087
Best Trade [%]                          36.464118
Worst Trade [%]                        -10.963919
Avg Winning Trade [%]                    10.09316
Avg Losing Trade [%]                    -4.336811


In [13]:
import numpy as np
import pandas as pd
import yfinance as yf
import vectorbt as vbt
from sklearn.linear_model import LinearRegression
import itertools

# Function to calculate Exponential Moving Average (EMA)
def calculate_ema(series, span=200):
    return series.ewm(span=span, adjust=False).mean()

# Function to calculate Hull Moving Average (HMA)
def calculate_hma(series, window=200):
    wma1 = series.rolling(window=window//2).mean()
    wma2 = series.rolling(window=window).mean()
    hma = (2 * wma1 - wma2).rolling(window=int(np.sqrt(window))).mean()
    return hma

# Function to calculate Percentage Price Oscillator (PPO)
def calculate_ppo(close, fast_span=12, slow_span=26):
    fast_ema = calculate_ema(close, span=fast_span)
    slow_ema = calculate_ema(close, span=slow_span)
    ppo = (fast_ema - slow_ema) / slow_ema * 100  # PPO as a percentage
    return ppo

# Function to calculate the Signal Line for PPO
def calculate_ppo_signal(ppo, signal_span=9):
    return ppo.ewm(span=signal_span, adjust=False).mean()

# Function to calculate linear regression trendline projection
def calculate_trendline_projection(df, window=200):
    slope = np.full(len(df), np.nan)

    for i in range(window, len(df)):
        y = df['Close'][i-window:i].values.reshape(-1, 1)
        x = np.arange(0, window).reshape(-1, 1)

        model = LinearRegression().fit(x, y)
        slope[i] = model.coef_[0][0]  # Get the slope of the linear regression

    return pd.Series(slope, index=df.index)

# Define the stock symbol and time period
symbol = 'VST'
start_date = '2019-01-01'
end_date = '2025-01-01'

# Download the data
df = yf.download(symbol, start=start_date, end=end_date)
df.columns = ['Close', 'High', 'Low', 'Open', 'Volume']

# Define function to optimize parameters
def optimize_parameters(hma_windows, fast_spans, slow_spans, ppo_signal_spans, trendline_windows):
    best_total_return = -np.inf
    best_params = None

    # Use itertools.product to generate all combinations of parameters
    param_combinations = itertools.product(hma_windows, fast_spans, slow_spans, ppo_signal_spans, trendline_windows)

    for hma_window, fast_span, slow_span, ppo_signal_span, trendline_window in param_combinations:
        # Calculate HMA for trend direction
        df['HMA'] = calculate_hma(df['Close'], window=hma_window)

        # Calculate the Linear regression slope
        df['Slope'] = calculate_trendline_projection(df, window=trendline_window)

        # Calculate PPO and PPO Signal line
        df['PPO'] = calculate_ppo(df['Close'], fast_span=fast_span, slow_span=slow_span)
        df['PPO_Signal'] = calculate_ppo_signal(df['PPO'], signal_span=ppo_signal_span)

        # Filter data for the test period (2020-2025)
        df_filtered = df[(df.index.year >= 2020) & (df.index.year <= 2025)]

        # Define Entry and Exit signals based on PPO and Moving Average Slope
        df_filtered['Entry'] = (df_filtered['Slope'] > 0) & (df_filtered['PPO'] > df_filtered['PPO_Signal']) & (df_filtered['Close'] > df_filtered['HMA'])
        df_filtered['Exit'] = (df_filtered['Slope'] < 0) & (df_filtered['PPO'] < df_filtered['PPO_Signal']) & (df_filtered['Close'] < df_filtered['HMA'])

        # Backtest using vectorbt
        portfolio = vbt.Portfolio.from_signals(
            close=df_filtered['Close'],
            entries=df_filtered['Entry'],
            exits=df_filtered['Exit'],
            init_cash=100_000,
            fees=0.001
        )

        # Track the Sharpe ratio
        total_return = portfolio.stats()['Total Return [%]']
        if total_return > best_total_return:
            best_total_return = total_return
            best_params = {
                'hma_window': hma_window,
                'fast_span': fast_span,
                'slow_span': slow_span,
                'ppo_signal_span': ppo_signal_span,
                'trendline_window': trendline_window
            }

    return best_params, best_total_return

# Define parameter grids
hma_windows = [10, 50, 100, 200]
fast_spans = range(5, 21, 5)
slow_spans = range(25, 51, 5)
ppo_signal_spans = range(6, 19, 3)
trendline_windows = [100, 150, 200]

# Optimize parameters
best_params, best_total_return = optimize_parameters(
    hma_windows, fast_spans, slow_spans, ppo_signal_spans, trendline_windows
)

print(f"Best parameters: {best_params}")
print(f"Best Total Return: {best_total_return}")

Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'omega_ratio' requires frequency to be set

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sortino_ratio' requires frequency to be set

<ipython-input-13-7866d28a49ff>:75: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-13-7866d28a49ff>:76: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/lo

Best parameters: {'hma_window': 10, 'fast_span': 20, 'slow_span': 30, 'ppo_signal_span': 18, 'trendline_window': 100}
Best Total Return: 587.9444798246941


<ipython-input-13-7866d28a49ff>:75: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-13-7866d28a49ff>:76: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sharpe_ratio' requires frequency to be set

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'calmar_ratio' requires frequency to be set

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/s

In [14]:
# Calculate 200 HMA for trend direction
df['HMA'] = calculate_hma(df['Close'], window=10)

# Calculate the Linear regression slope
df['Slope'] = calculate_trendline_projection(df, window=100)

# Calculate PPO and PPO Signal line
df['PPO'] = calculate_ppo(df['Close'], 20, 30)
df['PPO_Signal'] = calculate_ppo_signal(df['PPO'], 18)

# Filter data for the test period (2020-2025)
df = df[(df.index.year >= 2020) & (df.index.year <= 2025)]

# Define Entry and Exit signals based on PPO and Moving Average Slope
df['Entry'] = (df['Slope'] > 0) & (df['PPO'] > df['PPO_Signal']) & (df['Close'] > df['HMA'])
df['Exit'] = (df['Slope'] < 0) & (df['PPO'] < df['PPO_Signal']) & (df['Close'] < df['HMA'])

# Backtest using vectorbt
portfolio = vbt.Portfolio.from_signals(
    close=df['Close'],
    entries=df['Entry'],
    exits=df['Exit'],
    init_cash=100_000,
    fees=0.001
)

# Display performance metrics
print(portfolio.stats())

# Plot equity curve
portfolio.plot().show()

<ipython-input-14-e5649d5ece12>:15: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-14-e5649d5ece12>:16: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sharpe_ratio' requires frequency to be set

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'calmar_ratio' requires frequency to be set

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/s

Start                         2020-01-02 00:00:00
End                           2024-12-31 00:00:00
Period                                       1258
Start Value                              100000.0
End Value                           687944.479825
Total Return [%]                        587.94448
Benchmark Return [%]                   587.968758
Max Gross Exposure [%]                      100.0
Total Fees Paid                        685.359994
Max Drawdown [%]                        34.859269
Max Drawdown Duration                       290.0
Total Trades                                    4
Total Closed Trades                             3
Total Open Trades                               1
Open Trade PnL                      576189.132641
Win Rate [%]                            33.333333
Best Trade [%]                          30.032084
Worst Trade [%]                         -9.530365
Avg Winning Trade [%]                   30.032084
Avg Losing Trade [%]                    -7.262661


In [15]:
# Filter Test Years
df = df[(df.index.year >= 2020) & (df.index.year <= 2025)]

# Buy and Hold Performance Metrics
df_holding = df['Close']
pf = vbt.Portfolio.from_holding(df_holding, init_cash=100_000)
pf.stats()

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sharpe_ratio' requires frequency to be set

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'calmar_ratio' requires frequency to be set

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'omega_ratio' requires frequency to be set

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sortino_ratio' requires frequency to be set



,Close
Start,2020-01-02 00:00:00
End,2024-12-31 00:00:00
Period,1258
Start Value,100000.0
End Value,687968.757793
Total Return [%],587.968758
Benchmark Return [%],587.968758
Max Gross Exposure [%],100.0
Total Fees Paid,0.0
Max Drawdown [%],45.749981
